In [4]:
import sys
try:
    from termcolor import colored
except ImportError as e:
    !{sys.executable} -m pip install termcolor
    from termcolor import colored

In [5]:
def InstallIfNeeded(PackageName):
        !{sys.executable} -m pip install {PackageName} > /dev/null

# node printer
read db and generate a chain of Node obj printable as tree

In [14]:
class Node():
    def __init__(self, data):
        self.data = data
        self.children = []
        self.hasChildAuth= False
        self.auth=False
        self.parent= self
        
    def setAuth(self,auth):
        self.auth=auth
    
    def setHierarchicalAuth(self,auth):
        self.auth=auth
        for c in self.children:
            c.setHierarchicalAuth(auth)
    
    def setInverseHierarchicalAuth(self,auth,first=True):
        if not first:
            self.hasChildAuth=auth
        if self.parent!=self:
            self.parent.setInverseHierarchicalAuth(auth,False)
    
    def add_child(self, obj):
        if type(obj)==list:
            for o in obj:
                if o not in self.children:
                    o.parent=self
                    self.children.append(o)
        else:
            if obj not in self.children:
                obj.parent=self
                self.children.append(obj)
    
    def get_children(self):
        return self.children
    
    def get_child(self,data):
        for ch in self.children:
                if ch.data==data:
                    return ch

    def has_child(self,search):
        for ch in self.children:
            if ch.data==search:
                return True
        return False
    
    def print_child(self):
        sys.stdout.write("[")
        for c in self.children:
            print('{0},'.format(c.data),end='')
        sys.stdout.write("\b]")




root = Node('ROOT')
l11=Node('11')
l12=Node('12')
l13=Node('13')

l21=Node('21')
l22=Node('22')
l23=Node('23')

l24=Node('24')
l25=Node('25')
l26=Node('26')

l27=Node('27')
l28=Node('28')
l29=Node('29')

l31=Node("31")
l32=Node("32")

l11.add_child([l21,l22,l23])
l12.add_child([l24,l25,l26])
l29.add_child([l31,l32])
l13.add_child([l27,l28,l29])
root.add_child([l11,l12,l13])


## pretty print tree
creds to [Vallentin](https://vallentin.dev/2016/11/29/pretty-print-tree)

In [7]:
def pprint_tree(node, file=None, _prefix="", _last=True):
    print(_prefix, "`- " if _last else "|- ", colored(node.data,'green') if node.auth else colored( node.data,'red') , sep="", file=file)
    _prefix += "   " if _last else "|  "
    child_count = len(node.children)
    for i, child in enumerate(node.children):
        _last = i == (child_count - 1)
        pprint_tree(child, file, _prefix, _last)

In [8]:
#pprint_tree(root)

## Tree searcher
leve lorder Traversal algorithm to create a new tree with only the authorized resources<br>
```diff
+Input: node that i want to check auth on
+Output: all nodes that i have access to
```

In [2]:
from urllib.parse import urlparse
def deleteUnauth(root):
    queue = []
    if root.hasChildAuth==True or root.auth==True:
        queue.append(root)
    else:
        root=None
    while len(queue)>0:
        node = queue.pop()
        newChildren=[]
        for c in node.children:
            if c.auth==True or c.hasChildAuth==True:
                queue.append(c)
                newChildren.append(c)
        node.children=newChildren
    return root
def deleteUnrequested(root,res):
    p=urlparse(res)
    rootIRI= p.scheme+"://"+p.netloc
    lvls=list(filter(None,p.path.split('/')))
    x=root
    children=[]
    if x.data==rootIRI:
        for l in lvls:
            if len(x.children)>0:
                nextChild=None
                for c in x.children:
                    if c.data==l:
                        nextChild=c
                        x.children=[c]
                        break
                    if nextChild==None:
                        print("auth child not found")
                        return None
                else:
                    print("no more children")
                    return None
            x=nextChild 
        return root
    else:
        print("root not match")
        return None



## Graph creator using library
...why

In [10]:
# InstallIfNeeded("blockdiag")
from blockdiag import parser, builder, drawer
import blockdiag

In [11]:
def print_blockdiag(tree, parent=None):
    if not parent: print('blockdiag { orientation = portrait')
    for key in tree:
        if parent: print('   {} -> {};'.format(parent, key))
        print_blockdiag(tree[key], key)
    if not parent: print('}')

tree= blockdiag.parser.parse_string('''blockdiag {
  A -> B -> C;
  B -> D;
}''')
diagram = builder.ScreenNodeBuilder.build(tree)
draw = drawer.DiagramDraw('PNG', diagram, filename="foo.png")
draw.draw()
draw.save()